In [ ]:
from utils import *
from probabilistic_models import *

## Loading Datasets

In [13]:
df_1 = pd.read_excel(r"D:\Study\Python\Projects\Paul Project\vendor1.xlsx")
df_2 = pd.read_excel(r"D:\Study\Python\Projects\Paul Project\vendor2.xlsx")

In [14]:
data = pd.concat([df_1, df_2], ignore_index=True)
df,df_dict = df_processor(data)

In [15]:
X_values = df['Inverse_Temp'].values
Y_values = df['Mpa'].values

## Weibull Model

In [19]:
weibull = WeibullModel(X_values, Y_values)
weibull.intercept,weibull.slope, weibull.shape

(5.89755456074455, 0.02332859854794873, 25.755298576718157)

## LogNormal Model

In [20]:
lognormal = LognormalModel(X_values, Y_values)
lognormal.k, lognormal.m, lognormal.sigma

(5.859842293281658, 0.024157582221448693, 0.03864222350319222)

In [ ]:
# def plot_different_cdf(df_4,cdf,u,w, df,name, cdf_formula, model_name):
def plot_different_cdf(model,data_frames, cdf_values,name, cdf_formula, model_name):
    temperature_values = np.linspace(10, 600, 100)

    fig, ax = plt.subplots(figsize=(8,6))

    # for (name,df) in selected_files.items():
    #     ax.scatter(df["Temperature"], df["YS"], edgecolors='black', alpha=0.7, s=30, label=f"{name}")

    if st.checkbox(f"✔️ Show Different CDF values", value=True, key=f"{model_name}_cdf_check"):
        for i in range(len(cdf)):
            ys_predicted_cdf = cdf_formula(temperature_values, shape, u, w, cdf[i])
            ax.plot(temperature_values, ys_predicted_cdf, linestyle="-", linewidth=1, label=f"Predicted YS (CDF={cdf[i]})")

    var_cdf = st.slider("Select CDF value", min_value=0.01, max_value=0.99, value=0.5, step=0.01, key=f"{model_name}_slider")
    ys_predicted_cdf = cdf_formula(temperature_values, shape, u, w, var_cdf)
    ax.plot(temperature_values, ys_predicted_cdf, linestyle="-", linewidth=2, label=f"Predicted YS (Selected CDF={var_cdf})")

    ax.set_xlabel("Temperature (°C)", fontsize=12, fontweight="bold")
    ax.set_ylabel("Yield Stress (YS)", fontsize=12, fontweight="bold")
    ax.set_title("Yield Stress vs. Temperature Comparison", fontsize=14, fontweight="bold")
    ax.legend()
    st.pyplot(fig)

def plot_different_cdf(df_4,cdf,u,w, df,name, cdf_formula, model_name):
    shape = np.mean(df_4["Shape"])
    temperature_values = np.linspace(10, 600, 100)

    fig, ax = plt.subplots(figsize=(8,6))

    for (name,df) in selected_files.items():
        ax.scatter(df["Temperature"], df["YS"], edgecolors='black', alpha=0.7, s=30, label=f"{name}")

    if st.checkbox(f"✔️ Show Different CDF values", value=True, key=f"{model_name}_cdf_check"):
        for i in range(len(cdf)):
            ys_predicted_cdf = cdf_formula(temperature_values, shape, u, w, cdf[i])
            ax.plot(temperature_values, ys_predicted_cdf, linestyle="-", linewidth=1, label=f"Predicted YS (CDF={cdf[i]})")

    var_cdf = st.slider("Select CDF value", min_value=0.01, max_value=0.99, value=0.5, step=0.01, key=f"{model_name}_slider")
    ys_predicted_cdf = cdf_formula(temperature_values, shape, u, w, var_cdf)
    ax.plot(temperature_values, ys_predicted_cdf, linestyle="-", linewidth=2, label=f"Predicted YS (Selected CDF={var_cdf})")

    ax.set_xlabel("Temperature (°C)", fontsize=12, fontweight="bold")
    ax.set_ylabel("Yield Stress (YS)", fontsize=12, fontweight="bold")
    ax.set_title("Yield Stress vs. Temperature Comparison", fontsize=14, fontweight="bold")
    ax.legend()
    st.pyplot(fig)